In [13]:
import pandas as pd
import numpy as np
import json
import csv
from datetime import datetime

In [14]:
usersOld = pd.read_csv('data-live-week3/users_prod.csv')
restsOld = pd.read_csv('data-live-week3/restaurants_prod.csv')
# mealsOld = pd.read_csv('data-live/meals_prod.csv')
hospsOld = pd.read_csv('data-live-week3/hospitals_prod.csv')
# infosOld = pd.read_csv('data-live/mealinfo_prod.csv')
seedData = pd.read_csv('data-live-week3/seed_data.csv')

In [15]:
userDict = {}
restDict = {}
mealDict = {}
infoDict = {}
hospDict = {}
def userToDict(r):
    userDict[r.username] = {"id":r.id}
def restToDict(r):
    restDict[r.restName] = {"id":r.id, "userId":r.userId}
def mealToDict(r):
    mealDict[r.mealName.rstrip()] = {"id":r.id, "restName":r.restName, "userId":r.userId, "mealName" : r.mealName}
def infoToDict(r):
    infoDict[r.type] = {"id":r.id}
def hospToDict(r):
    hospDict[r.hospName] = {"id":r.id}

v = usersOld.apply(userToDict, axis=1)
v = restsOld.apply(restToDict, axis=1)
# v = mealsOld.apply(mealToDict, axis=1)
# v = infosOld.apply(infoToDict, axis=1)
v = hospsOld.apply(hospToDict, axis=1)

In [16]:
hospDict

{'Hospital of the University of Pennsylvania (HUP)': {'id': 'a4e68be8-f417-595a-b7f0-3aa7a7f23a2b'},
 'Penn Presbyterian Medical Center (PPMC)': {'id': 'c8e20bfc-f2ad-51a3-8f73-56488d3a1cc5'},
 'Pennsylvania Hospital (PAH, Pennsy)': {'id': 'add6685c-120d-5dc7-85a1-30d0895cfd9d'}}

In [17]:
# Convert the date correctly
def getDate(r):
    time = ''
    if(r['meal'] == 'Lunch'):
        time = '12:00:00-04'
    elif(r['hospital'] == 'Hospital of the University of Pennsylvania (HUP)'):
        time = '20:30:00-04'
    elif(r['meal'] == 'Dinner'):
        time = '20:00:00-04'
    else:
        print(r)
    date = ["%02d" % int(v) for v in r['date'].split('/')]
    date = "%s-%s-%s" % (date[2], date[0], date[1])
    return ("%s %s" % (date, time))

seedData['timeslot'] = seedData[['hospital', 'date', 'meal']].apply(getDate, axis=1)
seedData = seedData.fillna(value=False)

In [18]:
# np.unique(np.concatenate([seedData['mealDiet%i' %i].unique() for i in range(1, 5)]))

In [19]:
dietLookup = {
  "V" : "Vegan",
  "VG" : "Vegetarian",
  "GF" : "Gluten Free",
  "DF" : "Dairy Free",
  "Vegan": "Vegan",
  "Vegetarian" : "Vegetarian",
  "Dairy-Free": "Dairy Free",
  "Gluten-Free" : "Gluten Free"
}
allergenLookup = {
  'Milk/Dairy' : 'milk/dairy',
  'Milk/dairy' : "milk/dairy",
  'Eggs' : "eggs",
  'Eggs in aioli' : 'eggs',
  'Pumpkin seeds (seed allergy)' : "pumpkin seeds",
  'Sesame' : 'sesame',
  'Sesame Oil' : 'sesame',
  'Soy' : 'soy',
  'Dairy' : 'milk/dairy',
  'Tree nuts' : 'tree nuts',
  'Fish': 'fish',
  'Shellfish' : 'shellfish',
  'Wheat' : 'wheat',
  'Peanuts': 'peanuts'
}

In [20]:
# Build the restaurant struct
restStruct = {}
mealInfoStruct = {}

def dateCompare(t0, t1='2020-05-18 00:00:00-04'):
    v1 = datetime.strptime(t0, '%Y-%m-%d %H:%M:%S-04')
    v2 = datetime.strptime(t1, '%Y-%m-%d %H:%M:%S-04')
    return v1 > v2

def buildStruct(r):
    restName = r['restName']
    restStruct[restName] = restStruct.get(restName, {"mealIds": {}, "meals": [], "timeslots": [], "menus": []})
    # Create the timeslot and add to list
    timeslotId = str(("%s-%s-%s" % (restName, r['timeslot'], r['hospital'])).__hash__())
    timeslot = {
        "id": timeslotId,
        "hospitalId": hospDict[r['hospital']]['id'],
        "restaurantId": restDict[r['restName']]['id'],
        "date": r['timeslot'],
        "userId": restDict[r['restName']]['userId']
    }
    if dateCompare(r['timeslot']):
        restStruct[restName]['timeslots'].append(timeslot) 
    # Generate the meals/menus
    for i in range(1, 5):
        mealName = r["mealName%d" % i]
        if(mealName):
            mealName = mealName.rstrip()
            allergens, dietaryRestrictions = [], []
            mealType = r['meal'].lower().lstrip().rstrip()
            price = r['mealPrice%d' %i]
            mealInfoId = '%s - %0.2f' % (mealType, price) 
            if (not mealInfoId in mealInfoStruct):
                mealInfoStruct[mealInfoId] = {
                    "id" : mealInfoId,
                    "type" : mealInfoId,
                    "price" : price
                }
            
            if(r['mealAllergens%d' %i]):
                allergens = [v.lstrip().rstrip() for v in r['mealAllergens%d' %i].split(',')]
                allergens = [allergenLookup[a] for a in allergens]
            if(r['mealDiet%d' % i]):
                dietaryRestrictions = [v.lstrip().rstrip() for v in r['mealDiet%d' % i].split(',')]
                dietaryRestrictions = [dietLookup[d] for d in dietaryRestrictions]
        # Check if meal exists else add
            if(not mealName in mealDict):
                print(mealName, restName);
                meal = {
                    "name": mealName,
                    "description": r['mealDesc%d' %i],
                    "allergens" : allergens,
                    "dietaryRestrictions": dietaryRestrictions,
                    "mealinfoId": mealInfoStruct[mealInfoId]['id'],
                    "restaurantId": restDict[r['restName']]['id'],
                    "userId": restDict[r['restName']]['userId'],
                    "price": r['mealPrice%d' %i]
                }
                mealDict[mealName] = meal
                restStruct[restName]['meals'].append(meal)
            # Add the menu to the list
            menu = {
                "timeslotId": timeslotId,
                "mealinfoId": mealInfoStruct[mealInfoId]['id'],
                "mealName": mealName,
                "mealDescription": r['mealDesc%d' %i],
                "allergens" : allergens,
                "dietaryRestrictions": dietaryRestrictions,
                "userId": restDict[r['restName']]['userId'],
                "price": r['mealPrice%d' %i]
            }
            if dateCompare(r['timeslot']):
                restStruct[restName]['menus'].append(menu)
            
s = seedData.apply(buildStruct, axis=1)

('Southern Caesar Salad', 'Rex 1516')
('Strawberry Salad', 'Rex 1516')
('Smoked Pulled Pork Sandwich', 'Rex 1516')
('Blackened Shrimp Po\xe2\x80\x99Boy', 'Rex 1516')
('Cold Longevity Noodles with Roasted Chicken', 'Baology')
('Cold Longevity Noodles with Shredded Heritage Pork', 'Baology')
('Cold Longevity Noodles with Soy-Braised Egg', 'Baology')
('Chicken & Broccoli', 'Baology')
('Pork & Broccoli', 'Baology')
('Local Tofu & Broccoli', 'Baology')
('Ensalada Ynez', 'Cafe Ynez')
('Veggie burrito', 'Cafe Ynez')
('Chicken quesadilla', 'Cafe Ynez')
('Carnitas burrito', 'Cafe Ynez')
('Crispy Chicken Sandwich', 'On Point Bistro')
('VEGAN Housemade Black Bean Burger', 'On Point Bistro')
('Chicken Caesar Salad', 'On Point Bistro')
('Crab Cake with Cucumber & Cabbage Slaw, Horseradish Aioli', 'Pumpkin')
('Rigatoni with Golden Raisins, Calabrian Chili, Green Olive', 'Pumpkin')
('Mixed Veggies, Tabouli, Tahini, Chickpeas', 'Pumpkin')
('Porchetta Wrap', 'On Point Bistro')
('Veggie Pasta', 'On Poin

In [21]:
dataOut = json.dumps(restStruct, indent=2)
with open("update_data.json", "w") as outfile: 
    outfile.write(dataOut)
mealInfoOut = json.dumps(mealInfoStruct, indent=2)
with open("mi_update_data.json", "w") as outfile: 
    outfile.write(mealInfoOut) 

In [22]:
# Output the csvs
with open("timeslot_data.csv", "w") as outfile:
    writer = csv.writer(outfile)
    writer.writerow(['id', 'restaurantId', 'hospitalId', 'userId', 'date'])
    for restName in restStruct:
        for ts in restStruct[restName]['timeslots']:
            writer.writerow([ts['id'], ts['restaurantId'], ts['hospitalId'], ts['userId'], ts['date']])

In [23]:
# Output the csvs
with open("menu_data.csv", "w") as outfile:
    writer = csv.writer(outfile)
    writer.writerow(['timeslotId', 'mealinfoId', 'userId', 'mealName', 'mealDescription', 'allergens', 'dietaryRestrictions'])
    for restName in restStruct:
        for ts in restStruct[restName]['menus']:
            writer.writerow([ts['timeslotId'], ts['mealinfoId'], ts['userId'], ts['mealName'], ts['mealDescription'], ts['allergens'], ts['dietaryRestrictions']])
            

In [24]:
# Output the csvs
with open("meal_data.csv", "w") as outfile:
    writer = csv.writer(outfile)
    writer.writerow(['restaurantId', 'mealinfoId', 'userId', 'mealName', 'mealDescription', 'allergens', 'dietaryRestrictions'])
    for restName in restStruct:
        for ts in restStruct[restName]['meals']:
            writer.writerow([ts['restaurantId'], ts['mealinfoId'], ts['userId'], ts['name'], ts['description'], ts['allergens'], ts['dietaryRestrictions']])
            